In [54]:
# modules imports
import nltk
# nltk.download('gutenberg')
from nltk.corpus import gutenberg
import re
import string
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import numpy as np

In [2]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [3]:
hamlet = gutenberg.words('shakespeare-hamlet.txt')

In [7]:
print(len(hamlet))
print(hamlet[:40])
print(type(hamlet))

37360
['[', 'The', 'Tragedie', 'of', 'Hamlet', 'by', 'William', 'Shakespeare', '1599', ']', 'Actus', 'Primus', '.', 'Scoena', 'Prima', '.', 'Enter', 'Barnardo', 'and', 'Francisco', 'two', 'Centinels', '.', 'Barnardo', '.', 'Who', "'", 's', 'there', '?', 'Fran', '.', 'Nay', 'answer', 'me', ':', 'Stand', '&', 'vnfold', 'your']
<class 'nltk.corpus.reader.util.StreamBackedCorpusView'>


In [21]:
# Cleaning data

# removing non-alphabetical symbols and empty strings
regex = re.compile('[^a-zA-Z ]')
hamlet_clean = [regex.sub('', x) for x in hamlet if regex.sub('', x)]

<class 'list'>


In [40]:
# on-hot encoding letters

lowercase = list(string.ascii_lowercase)
uppercase = list(string.ascii_uppercase)
all_letters = lowercase + uppercase
all_letters.append(" ")

letters_encoded = {}
for index, letter in enumerate(all_letters):
    encode = [0]*len(all_letters)
    encode[index] = 1
    letters_encoded.update({letter:encode})

# inverse dictionary
letters_encoded_inv = {tuple(v): k for k, v in letters_encoded.items()}


In [50]:
# creating 40 symbol sequences

# converting list to one string
hamlet_string = " ".join(hamlet_clean)

# encoding by mapping
hamlet_string_encoded = [letters_encoded[l] for l in hamlet_string]
# print(hamlet_string_encoded[:20])

# creating sequences of 40 symbols
hamlet_sequences_encoded = []
y_41th_symbol = []
for index, symbol in enumerate(hamlet_string_encoded):
    if index <= (len(hamlet_string_encoded)-41):
        sequence = hamlet_string_encoded[index:index+40]
        next_symbol = hamlet_string_encoded[index+40]
        hamlet_sequences_encoded.append(sequence)
        y_41th_symbol.append(next_symbol)



In [55]:
# split data
x_train, x_test, y_train, Y_test = train_test_split(hamlet_sequences_encoded, y_41th_symbol, test_size= 0.2, random_state=2)

x_train = np.array(x_train)
y_train = np.array(y_train)

print("x_train shape", x_train.shape)
print("y_train shape", y_train.shape)

x_train shape (122023, 40, 53)
y_train shape (122023, 53)


In [ ]:
# Building LSTM model

pool_size = len(all_letters)

model = Sequential()
model.add(LSTM(50, input_shape=(40, pool_size)))
model.add(Dropout(0.2))
model.add(Dense(pool_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
